In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

%matplotlib inline
custom_style = {'axes.labelcolor': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white'}
sns.set_style("darkgrid", rc=custom_style)
sns.set_context("notebook")
plt.style.use('dark_background')
plt.rcParams["font.size"] = 18
np.random.seed(123)

In [2]:
df = pd.read_csv('./data/ch6_2_1.csv', header=None)
df = df.T
df.columns = ['x1', 'x2']
# x1 = np.array(df['x1']).astype(np.float)
# x2 = np.array(df['x2']).astype(np.float)

In [3]:
stan_model = """
    data{
        int N1;
        int N2;
        real X1[N1];
        real X2[N2];
    }
    
    parameters{
        real mu1;
        real mu2;
        real<lower=0> sigma1;
#        real<lower=0> sigma2;
    }
    
    model{
        for(n in 1:N1){
            X1[n] ~ normal(mu1, sigma1);
        }
        for(n in 1:N2){
            X2[n] ~ normal(mu2, sigma1);
#            X2[n] ~ normal(mu2, sigma2);
        }
#        X1 ~ normal(mu1, sigma1);
#        X2 ~ normal(mu2, sigma2);

    }
    
    generated quantities{
        real mu_delta;
        real<lower=0, upper=1> u_0;
        real<lower=0, upper=1> u_1;

        mu_delta <- mu2 - mu1;
        u_0 <- step(mu_delta - 0);
        u_1 <- step(mu_delta - 1);

    }
"""

In [4]:
sm = pystan.StanModel(model_code = stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_04ff8c98382d23d731a8902ef5b039e1 NOW.


In [6]:
stan_data = {"N1": df.shape[0], "N2": df.shape[0], "X1": df['x1'], "X2": df['x2']}
fit = sm.sampling(data=stan_data, iter=11000, warmup=1000, chains=3, seed=1234)

/Users/yugi/.pyenv/versions/anaconda-4.0.0/envs/py36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [7]:
fit

Inference for Stan model: anon_model_04ff8c98382d23d731a8902ef5b039e1.
3 chains, each with iter=11000; warmup=1000; thin=1; 
post-warmup draws per chain=10000, total post-warmup draws=30000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu1       30.84  1.9e-3   0.31  30.22  30.63  30.84  31.04  31.45  27120    1.0
mu2       31.98  1.9e-3   0.32  31.36  31.77  31.98  32.19   32.6  27198    1.0
sigma1      1.4  1.1e-3   0.17   1.12   1.28   1.38    1.5   1.77  23615    1.0
mu_delta   1.15  2.7e-3   0.44   0.27   0.85   1.15   1.44   2.01  27352    1.0
u_0        0.99  4.7e-4   0.07    1.0    1.0    1.0    1.0    1.0  24717    1.0
u_1        0.63  2.9e-3   0.48    0.0    0.0    1.0    1.0    1.0  28003    1.0
lp__     -32.34    0.01   1.28 -35.67  -32.9 -32.01 -31.41  -30.9  13281    1.0

Samples were drawn using NUTS at Tue Aug 14 08:29:10 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduc